# Классификация изображений с помощью свёрточной нейронной сети (CNN)

Импортируем необходимые библиотеки

In [3]:
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Распакуем данные

In [4]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


Функция load_data(batch) - метод для передора вскх пакетов в cifar-10, из разбора и загрузки в массивы X и Y.

X - массив, содержащий вск данные изображения из загруженных пакетов.

Y - массив соответствующих меток для всех изображений в X.



In [5]:
def load_data(batch):
    data = unpickle('/content/gdrive/MyDrive/cifar/' + batch)
    X = data[b'data']
    Y = data[b'labels']
    for i in range(2,6):
        data = unpickle('/content/gdrive/MyDrive/cifar/data_batch_'+str(i))
        X = np.concatenate((X,data[b'data']))
        Y = np.concatenate((Y,data[b'labels']))
    return X,Y

Имрортируем torch и подготовим данные

In [6]:
# Импортируем необходимые библиотеки PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


Подготовим данные

In [7]:
X,Y = load_data('data_batch_1')

# Изменение формы данных: изображения CIFAR-10 имеют размер 32x32 пикселей с 3 каналами (RGB).
# -1 в reshape означает автоматическое вычисление размера этого измерения
X = X.reshape(-1, 3, 32, 32) / 255.0

# Преобразование меток в тензор PyTorch типа long (который используется для целочисленных меток)
Y = torch.tensor(Y).long()

# Преобразование объектов в тензор PyTorch с типом float32
X = torch.tensor(X, dtype=torch.float32)

# Создание TensorDataset, который представляет собой набор данных, обертывающий тензоры
dataset = TensorDataset(X, Y)

# DataLoader для перебора набора данных с указанным размером пакета
# shuffle=True для перемешивания набора данных каждую эпоху
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)

Определим CNN

In [8]:
# # Определить класс CNN, наследующий от nn.Module
class SimpleCNN(nn.Module):
    # Конструктор для определения слоев
    def __init__(self):
        super(SimpleCNN, self).__init__()

        # Первый сверточный слой с 32 фильтрами размера 3x3 , шаг 1 и заполнение 1
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)

        # Максимальный слой пула с окном 2x2 и шагом 2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # Второй сверточный слой с 64 фильтрами размера 3x3, шаг 1 и заполнение 1
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)

        # Полносвязный слой, 64 * 8 * 8 — это сглаженный размер свернутых изображений
        self.fc1 = nn.Linear(64 * 8 * 8, 128)

        # Второй полносвязный слой, который выводит 10 классов (для CIFAR-10 )
        self.fc2 = nn.Linear(128, 10)

    # Затем для CNN определяется прямой метод, который определяет, как данные изображения перемещаются по слоям для создания прогнозов классов
    # Прямой проход через сеть
    def forward(self, x):
        # Применяем первую свертку, за которой следует активация ReLU и объединение в пул
        x = self.pool(F.relu(self.conv1(x)))

        # Применяем вторую свертку, а затем Активация и объединение ReLU
        x = self.pool(F.relu(self.conv2(x)))

        # Сглаживаем тензор для полностью связного слоя
        x = x.view(-1, 64 * 8 * 8)

        # Применяем первый полносвязный слой с активацией ReLU
        x = F.relu(self.fc1(x))

         # Применяем второй полносвязный слой
        x = self.fc2(x)

        # Возвращаем выходные данные
        return x

Теперь определим цикл, который мы будем использовать для обучения нашей модели.

In [26]:
# Check for GPU availability and set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Создание экземпляра модели CNN и перенос его на выбранное устройство (GPU или CPU)
model = SimpleCNN().to(device)

# Определение функции потерь (для классификации используется CrossEntropyLoss)
criterion = nn.CrossEntropyLoss()

# Определение оптимизатора (стохастический градиентный спуск) со скоростью обучения и импульсом
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Цикл обучения
for epoch in range(15):

    for i, data in enumerate(train_loader, 0):

        inputs, labels = data[0].to(device), data[1].to(device)

        # Обнуляем градиенты
        optimizer.zero_grad()

        # Переход вперед: вычисляем прогнозируемые выходные данные
        outputs = model(inputs)

        # Вычисляем потери
        loss = criterion(outputs, labels)

        # Perform backpropagation: compute gradient of the loss with respect to model parameters
        loss.backward()

        # Perform optimization step (parameter update)
        optimizer.step()

    # Print loss after each epoch
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Print a message when training is finished
print("Finished Training")

Epoch 1, Loss: 2.029019594192505
Epoch 2, Loss: 1.918224573135376
Epoch 3, Loss: 1.7451707124710083
Epoch 4, Loss: 1.6826088428497314
Epoch 5, Loss: 1.8255927562713623
Epoch 6, Loss: 0.9288021922111511
Epoch 7, Loss: 1.4745497703552246
Epoch 8, Loss: 1.14347243309021
Epoch 9, Loss: 1.0831741094589233
Epoch 10, Loss: 1.1212232112884521
Epoch 11, Loss: 1.061507225036621
Epoch 12, Loss: 0.8533371090888977
Epoch 13, Loss: 1.0768177509307861
Epoch 14, Loss: 1.5244405269622803
Epoch 15, Loss: 0.9759458899497986
Finished Training


Сохраняем обученную модель

In [27]:
# Сохраняем всю модель
# torch.save(model, 'cifar10_model_full.pth')

# Сохраняем только словарь состояний
torch.save(model.state_dict(), 'cifar10_model_state_dict.pth')

# Сохраним на гугл диск
! touch "/content/gdrive/MyDrive/cifar/cifar10_model_state_dict.pth"

In [28]:
# Define the model (must be the same architecture as the trained model)
model = SimpleCNN()

# Load the state dictionary into the model (assuming state dictionary was saved)
model.load_state_dict(torch.load('cifar10_model_state_dict.pth'))
# Set the model to evaluation mode
model.eval()

SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

Тестовые данные

In [29]:
# Unpickle the test data
test_data = unpickle('/content/gdrive/MyDrive/cifar/test_batch')

Подсчитаем точность модели на тестовом наборе данных

In [30]:
# Извлекаем объекты (X_test) и метки (Y_test) из тестовых данных
X_test = test_data[b'data']
Y_test = test_data[b'labels']

# Изменяем форму и нормализовываем тестовые данные
# Тестовые изображения CIFAR-10 имеют размер 32x32 пикселя с 3 цветовых канала (RGB)
X_test = X_test.reshape(-1, 3, 32, 32) / 255.0

# Преобразование тестовых данных в тензоры PyTorch
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test = torch.tensor(Y_test).long()

# Создание DataLoader для тестовых данных
test_dataset = TensorDataset(X_test, Y_test)
test_loader = DataLoader(test_dataset, batch_size=64)

# Загружаем обученную модель (при условии, что state dictionary сохранен)
model = SimpleCNN()
model.load_state_dict(torch.load('cifar10_model_state_dict.pth'))
model.eval()

# Оценка модели на тестовых данных
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct / total}%')

Accuracy of the network on the test images: 61.82%


Определим методы загрузки и обработки, которые могут использовать мой тип модели для классификации моих изображений

In [31]:
import torchvision.transforms as transforms
from PIL import Image

# Функция для загрузки и предварительной обработки изображения
def process_image(image_path):
    # Преобразования изображений
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # CIFAR-10 normalization
    ])

   # Загрузка изображения
    image = Image.open(image_path)
    image = image.convert('RGB')  # Convert to RGB

    # Примененение преобразования
    return transform(image)

# Функция для классификации изображения
def classify_image(image_path, model_path):
    # Загружаем и обрабатываем изображение
    image = process_image(image_path)

    # Добавляем дополнительное измерение пакета, поскольку PyTorch рассматривает все входные данные как пакеты
    image = image.unsqueeze(0)

    # Загружаем обученную модель
    model = SimpleCNN()
    model.load_state_dict(torch.load(model_path))
    model.eval()

    # Классифицируем изображение
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    # Преобразование прогнозируемого индекса класса в имя класса (если имена классов доступны)
    # Для CIFAR-10 классы:
    classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    predicted_class = classes[predicted[0]]

    return predicted_class

Определяем свои собственные изображения

In [32]:
image_path = '/content/gdrive/MyDrive/cifar/MC21.jpg'
model_path = 'cifar10_model_state_dict.pth'
predicted_class = classify_image(image_path, model_path)
print(f'Predicted class: {predicted_class}')

Predicted class: airplane
